In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import sklearn
#import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:

#Do this just once - No need to do any more
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials 
#Authenticate and create the PyDrive client.


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

 

In [ ]:
id = "1EcWRAgZcqNwH9g9Ko-TkZvAZQSaTeZOu" # 30P
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('30P.csv')  
P = pd.read_csv('30P.csv', header=None)

id = "1pfx3L4ivHItHUlgvQKlmZZPVWS14rt81" # 30Q
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('30Q.csv')  
Q = pd.read_csv('30Q.csv', header=None)

id = "1Dq9lxJ5AI-IzX2lSmeENdV7iUvWam4Ev" # 30V
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('30V.csv')  
V = pd.read_csv('30V.csv', header=None)

id = "1P1h6FFRnFN0YyXP9S-4nJt5uYVV_KaIf" # 30Va
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('30Va.csv') 
Va = pd.read_csv('30Va.csv', header=None)

In [ ]:
print(P.shape)
print(Q.shape)
print(V.shape)
print(Va.shape)

(500, 30)
(500, 30)
(500, 30)
(500, 30)


In [ ]:
num_train = P.shape[0]
num_bus = P.shape[1]
X = Va.join(V, lsuffix='_V', rsuffix='_Va')
Y = P.join(Q, lsuffix='_P', rsuffix='_Q')



In [ ]:
#Train_test splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import mean_absolute_error
def mean_absolute_percentage_error(y_true, y_pred): 
  num_train = y_true.shape[0]
  num_bus = y_true.shape[1]
  error = np.zeros(num_train)
  for i in range(num_train):
    y1 = y_true.to_numpy()[i,:]
    y2 = y_pred.to_numpy()[i,:]
    error[i] =  np.mean(np.abs((y1 - y2) / y1)) * 100
  res = error.mean()
  return res
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    #return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



In [ ]:
from keras.models import Sequential # Used to build model
from keras.layers import Dense # Type of layer
from keras.optimizers import Adam # Optimization technique
from keras.layers import Dropout # For tuning the neural network
from keras import regularizers # For regularization

In [ ]:
# Variation
model1 = Sequential()
model1.add(Dense(num_bus, input_dim=num_bus*2, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model1.add(Dense(num_bus*2, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model1.add(Dense(num_bus*2, activation='relu')) 
# compile model
adam = Adam(lr=0.001)
mape = tf.keras.losses.MeanAbsolutePercentageError()
m = tf.keras.metrics.MeanAbsolutePercentageError(
    name="mean_absolute_percentage_error", dtype=None
)
model1.compile(loss=mape, optimizer=adam, metrics=[m])

model1.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                1830      
_________________________________________________________________
dense_1 (Dense)              (None, 60)                1860      
_________________________________________________________________
dense_2 (Dense)              (None, 60)                3660      
Total params: 7,350
Trainable params: 7,350
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Hyper parameter tuning
from sklearn.model_selection import GridSearchCV

from keras.wrappers.scikit_learn import KerasRegressor

def create_model(optimizer='adam'):
      model =Sequential()
      model.add(Dense(num_bus*2,input_dim =num_bus*2,kernel_initializer='normal',kernel_regularizer=regularizers.l2(0.001),activation='linear'))
      model.add(Dense(num_bus*2,kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='linear'))
      model.compile(loss=mean_absolute_percentage_error, optimizer=optimizer, metrics=[mean_absolute_percentage_error])
      return model


In [ ]:
estimator1 = KerasRegressor(build_fn=create_model, epochs=100, batch_size=32, verbose=0)
optimizers = [ 'Adagrad', 'Adam']
param_grid = dict(optimizer=optimizers)
grid = GridSearchCV(estimator=estimator1, param_grid=param_grid,scoring=mean_absolute_percentage_error,cv=3)


In [ ]:
grid_result = grid.fit(X_train,y_train)
# summarize results

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: in user code:

    /usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-8-5f142e8d0010>:6 mean_absolute_percentage_error  *
        for i in range(num_train):
    /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/operators/py_builtins.py:365 range_  **
        return _py_range(start_or_stop, stop, step)
    /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/operators/py_builtins.py:390 _py_range
        return range(start_or_stop)

    TypeError: 'NoneType' object cannot be interpreted as an integer


  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: 

TypeError: ignored

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.000000 using {'optimizer': 'Adagrad'}


In [ ]:
'''#Functional Model
from keras.models import Model
from keras.layers import Input
input1 =Input(shape=(num_bus*2,))
hidden1  = Dense(units =num_bus, activation ='relu')(input1)
#hidden2 =  Dense(units =120,activation ='relu')(hidden1)
output =  Dense(units =num_bus*2,activation ='relu')(hidden1)

model_functional = Model(inputs =input1, outputs =output)
model_functional.compile(loss=mape, optimizer='adam', metrics=[m])
model_functional.summary()'''


"#Functional Model\nfrom keras.models import Model\nfrom keras.layers import Input\ninput1 =Input(shape=(num_bus*2,))\nhidden1  = Dense(units =num_bus, activation ='relu')(input1)\n#hidden2 =  Dense(units =120,activation ='relu')(hidden1)\noutput =  Dense(units =num_bus*2,activation ='relu')(hidden1)\n\nmodel_functional = Model(inputs =input1, outputs =output)\nmodel_functional.compile(loss=mape, optimizer='adam', metrics=[m])\nmodel_functional.summary()"

In [ ]:
'''#initializing neural network
model1 = Sequential()
model1.add(Dense(num_bus, input_dim=num_bus*2, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model1.add(Dense(num_bus*2, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
# model1.add(Dense(2, activation='softmax')) #in the output we have two neurons as this is the categorical model
# compile model
adam = Adam(lr=0.001)
mape = tf.keras.losses.MeanAbsolutePercentageError()
m = tf.keras.metrics.MeanAbsolutePercentageError(
    name="mean_absolute_percentage_error", dtype=None
)
model1.compile(loss=mape, optimizer='adam', metrics=[m])

model1.summary()
'''

'#initializing neural network\nmodel1 = Sequential()\nmodel1.add(Dense(num_bus, input_dim=num_bus*2, kernel_initializer=\'normal\', kernel_regularizer=regularizers.l2(0.001), activation=\'relu\'))\nmodel1.add(Dense(num_bus*2, kernel_initializer=\'normal\', kernel_regularizer=regularizers.l2(0.001), activation=\'relu\'))\n# model1.add(Dense(2, activation=\'softmax\')) #in the output we have two neurons as this is the categorical model\n# compile model\nadam = Adam(lr=0.001)\nmape = tf.keras.losses.MeanAbsolutePercentageError()\nm = tf.keras.metrics.MeanAbsolutePercentageError(\n    name="mean_absolute_percentage_error", dtype=None\n)\nmodel1.compile(loss=mape, optimizer=\'adam\', metrics=[m])\n\nmodel1.summary()\n'

In [ ]:
# fit the model1(categorical one) to the training data
history1=model1.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=100, batch_size=32) 


Epoch 1/100
13/13 [==============================] - 0s 16ms/step - loss: 65.7463 - mean_absolute_percentage_error: 65.7295 - val_loss: 65.7815 - val_mean_absolute_percentage_error: 65.7647
Epoch 2/100
13/13 [==============================] - 0s 5ms/step - loss: 65.7544 - mean_absolute_percentage_error: 65.7376 - val_loss: 65.7112 - val_mean_absolute_percentage_error: 65.6943
Epoch 3/100
13/13 [==============================] - 0s 5ms/step - loss: 65.7429 - mean_absolute_percentage_error: 65.7261 - val_loss: 65.7307 - val_mean_absolute_percentage_error: 65.7138
Epoch 4/100
13/13 [==============================] - 0s 5ms/step - loss: 65.7457 - mean_absolute_percentage_error: 65.7288 - val_loss: 65.7146 - val_mean_absolute_percentage_error: 65.6978
Epoch 5/100
13/13 [==============================] - 0s 5ms/step - loss: 65.7510 - mean_absolute_percentage_error: 65.7342 - val_loss: 65.7950 - val_mean_absolute_percentage_error: 65.7782
Epoch 6/100
13/13 [==============================] - 0

In [ ]:
#history2=model_functional.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=100, batch_size=32) 

Epoch 1/100
13/13 [==============================] - 0s 17ms/step - loss: 67.8687 - mean_absolute_percentage_error: 67.8687 - val_loss: 529.4394 - val_mean_absolute_percentage_error: 529.4394
Epoch 2/100
13/13 [==============================] - 0s 4ms/step - loss: 67.8828 - mean_absolute_percentage_error: 67.8828 - val_loss: 529.4883 - val_mean_absolute_percentage_error: 529.4883
Epoch 3/100
13/13 [==============================] - 0s 4ms/step - loss: 67.8883 - mean_absolute_percentage_error: 67.8883 - val_loss: 529.4471 - val_mean_absolute_percentage_error: 529.4471
Epoch 4/100
13/13 [==============================] - 0s 5ms/step - loss: 67.8621 - mean_absolute_percentage_error: 67.8621 - val_loss: 529.4592 - val_mean_absolute_percentage_error: 529.4592
Epoch 5/100
13/13 [==============================] - 0s 4ms/step - loss: 67.8828 - mean_absolute_percentage_error: 67.8828 - val_loss: 529.5005 - val_mean_absolute_percentage_error: 529.5005
Epoch 6/100
13/13 [=========================

In [ ]:
import pickle 
import os
pkl_filename = "/content/ann_model.pkl"
if os.path.exists(pkl_filename):
        os.remove(pkl_filename)
with open(pkl_filename, 'wb') as file:
      pickle.dump(model1, file)  

            

AttributeError: ignored

In [ ]:
y_test.shape

(100, 60)

In [ ]:
from sklearn.metrics import mean_absolute_error
eg = y_test.shape[0]
# results = a = [0 for x in range(eg)]
for i in range(eg):
    pred = model1.predict(X_test.iloc[[i]])
    act = y_test.iloc[[i]]
    results[i] = mean_absolute_error(pred,act)
    
import statistics
statistics.mean(results) 


0.034969797551911534